In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torchvision
import pydicom
import pickle
import glob
import sklearn
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression

import seaborn as sns
from datetime import datetime

from sentence_transformers import SentenceTransformer

from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import time
import os
import copy
from pprint import pprint
from fastai.basics import delegates
from fastcore.parallel import parallel
from fastcore.utils import gt
from fastcore.foundation import L

from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue
import sys
import importlib
import warnings
warnings.filterwarnings("ignore")

/Users/cmm/opt/anaconda3/envs/aipi540/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip install pandas==1.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 11.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.0
    Uninstalling pandas-2.0.0:
      Successfully uninstalled pandas-2.0.0


In [4]:
# loading the data back from pickled dataframes
with open('../data/fusion_train.pkl', 'rb') as file:
    ftrain = pickle.load(file)

with open('../data/fusion_val.pkl', 'rb') as file:
    fval = pickle.load(file)

with open('../data/fusion_test.pkl', 'rb') as file:
    ftest = pickle.load(file)

In [5]:
ftrain

,pixel_preds,true,pixel_probs,patientID,series_description,contrast,plane,meta_preds,meta_probs,nlp_preds,nlp_probs
0,8,8,"[8.453496775473468e-06, 3.699102671816945e-05, 4.4237281144887675e-06, 4.551899110083468e-06, 0.9991437196731567, 0.0005227549700066447, 8.057334525801707e-06, 2.3983184291864745e-06, 4.916629677609308e-06, 1.0506399121368304e-05, 2.6590940251480788e-05, 3.911186013283441e-06, 2.7793970730272122e-05, 2.8801745429518633e-05, 6.744251732015982e-05, 1.0035731065727305e-05, 3.4046659038722282e-06, 7.080972864059731e-06, 7.819172606104985e-05]",104,ax diff_TRACEW,1,ax,8,"[0.13959238441186997, 0.0, 0.0, 0.002532022639261245, 0.8371638490976586, 0.013689810144576497, 4.787437763309077e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0007161233789062767, 0.005262991763510616, 0.0009949441865838582, 0.0, 0.0, 0.0, 0.0]",8,"[0.020583117219662772, 0.010903398861520041, 0.0009219695432471948, 0.001186827366113606, 0.8919651283538699, 0.023230189183809247, 0.0003882503184114099, 0.01115494172376811, 0.0026917216997651666, 0.0008547114874260082, 0.005626948501018502, 0.0003480571803900264, 0.004635548008826763, 0.002395440110027482, 0.002171796117076895, 0.007222901655572152, 0.00034788598859700095, 0.0038803619170794737, 0.009490804763818377]"
1,8,8,"[2.6521377094468335e-06, 8.239035196311306e-06, 2.173357188439695e-06, 1.107213051909639e-06, 0.9997029900550842, 0.00018724275287240744, 2.2190652089193463e-06, 7.945341735648981e-07, 1.5392799923574785e-06, 3.515505568429944e-06, 3.850110715575283e-06, 1.3153871805116069e-06, 3.4736724046524614e-05, 7.983376235642936e-06, 1.6097115803859197e-05, 4.0718127820582595e-06, 1.2490519338825834e-06, 1.037434321915498e-05, 7.827717126929201e-06]",104,ax diff_TRACEW,1,ax,8,"[0.13959238441186997, 0.0, 0.0, 0.002532022639261245, 0.8371638490976586, 0.013689810144576497, 4.787437763309077e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0007161233789062767, 0.005262991763510616, 0.0009949441865838582, 0.0, 0.0, 0.0, 0.0]",8,"[0.020583117219662772, 0.010903398861520041, 0.0009219695432471948, 0.001186827366113606, 0.8919651283538699, 0.023230189183809247, 0.0003882503184114099, 0.01115494172376811, 0.0026917216997651666, 0.0008547114874260082, 0.005626948501018502, 0.0003480571803900264, 0.004635548008826763, 0.002395440110027482, 0.002171796117076895, 0.007222901655572152, 0.00034788598859700095, 0.0038803619170794737, 0.009490804763818377]"
2,16,16,"[8.214457193389535e-05, 4.448040635907091e-05, 0.00016938522458076477, 4.597333827405237e-06, 2.931377457571216e-05, 0.00023740004689898342, 2.7919777494389564e-05, 1.587046244821977e-05, 0.0018987013027071953, 0.0002371210721321404, 0.00013228735770098865, 2.963991937576793e-05, 0.9491598010063171, 2.297530500072753e-06, 1.24157977552386e-05, 0.00024860951816663146, 3.45324951922521e-05, 0.04661918431520462, 0.0010144119150936604]",104,ax equilibrium,1,ax,16,"[0.0, 0.07347647881627575, 0.05536427711074582, 0.0, 0.0, 8.790049663780601e-06, 0.0, 0.00045239535937176793, 0.1127768828824028, 0.06864033122807019, 0.0004762623284278362, 0.0, 0.3350741051627684, 8.325808153216392e-05, 4.120687347588036e-05, 0.0004732497896584059, 1.1968594408272692e-05, 0.3121743026895439, 0.040946491033655455]",16,"[0.01010469118565345, 0.023720291059151646, 0.004030169855110401, 0.001347889236481563, 0.011807272878532152, 0.014338979151129886, 0.0005932208909441949, 0.009543675821096585, 0.05088408064135952, 0.0014002964901355017, 0.014362836710930965, 0.0005589913858506072, 0.8214210065712495, 0.0029564122952476218, 0.0032360335551993714, 0.013837046538673212, 0.0005586499437811647, 0.0049143071961676756, 0.010384148593304906]"
3,9,9,"[1.407535137332161e-06, 4.045129571750294e-06, 1.170156110674725e-06, 1.5774699022585992e-06, 9.902709280140698e-05, 0.9998393058776855, 1.0094064464283292e-06, 5.267510800877062e-07, 3.2990649287967244e-06, 3.343656317156274e-06, 4.074325715919258e-06, 8.580621511100617e-07, 1.4936608749849256e-06, 1.4751939261259395e-06, 6.3492607296211645e-06, 4.13683392253

In [6]:
!pip install --upgrade pandas

  Using cached pandas-2.0.0-cp311-cp311-macosx_10_9_x86_64.whl (11.6 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.0
    Uninstalling pandas-1.5.0:
      Successfully uninstalled pandas-1.5.0


In [9]:
ftest.to_pickle('../data/fusion_test.pkl')
fval.to_pickle('../data/fusion_val.pkl')
ftrain.to_pickle('../data/fustion_train.pkl')
